# Mask R-CNN - Nephrology Multi-Inference
This is an custom version of [Mask R-CNN - Train cell nucleus Dataset](https://colab.research.google.com/github/navidyou/Mask-RCNN-implementation-for-cell-nucleus-detection-executable-on-google-colab-/blob/master/mask_RCNN_cell_nucleus_google_colab.ipynb) for Google Colab. 

If using this notebook on Google Colab, GPU/TPU might not be used due to version of TensorFlow.

## Google Colab Only

Execute only if using this notebook on Google Colab (installing compatible librairies and getting files needed). Errors might appear, do not worry about this.

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
print("Executing in Google Colab" if IN_COLAB else "Executing locally")

In [ ]:
if IN_COLAB:
    import os
    !pip install -q scipy==1.1
    !pip install -q tensorflow==1.7
    !pip install -q keras==2.1.6
    GITHUB_REPO = "https://raw.githubusercontent.com/AdrienJaugey/Custom-Mask-R-CNN-for-kidney-s-cell-recognition/master/"
    files = ['mrcnn/config.py', 'mrcnn/utils.py', 'mrcnn/model.py', 
             'mrcnn/visualize.py', 'datasetTools/datasetDivider.py',
             'datasetTools/datasetWrapper.py', 'datasetTools/AnnotationExporter.py', 
             'datasetTools/ASAPExporter.py', 'datasetTools/LabelMeExporter.py', 'nephrology.py']
    for fileToDownload in files:
        url = GITHUB_REPO + fileToDownload
        !wget -qN $url
        if '/' in fileToDownload:
            destDir = fileToDownload.split('/')[0]
            fileName = fileToDownload.split('/')[1].replace('/', '')
            os.makedirs(destDir, exist_ok=True)
            !mv $fileName $fileToDownload

### Connecting to Google Drive

The first time this cell is executed, a link should appear, asking you to accept to give access to files of a google account. 
1.   **Follow the link**;
2.   **Choose the account** you want to link;
3.   **Accept**;
4.   **Copy the key** Google gave you;
5.   **Paste the key in the text field** that appeared below the first link you used,
6.   **Press ENTER**.

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import shutil
    shutil.rmtree('sample_data/', ignore_errors=True)

###Retrieving your image

Choose how to get your image from the following list on the right   
Use ```.jp2``` or ```.png``` images only !


In [ ]:
if IN_COLAB:
    howToGetImage = "From Google Drive" #@param ["Upload", "From Google Drive"]

#### By upload

In [ ]:
if IN_COLAB and howToGetImage == "Upload":
    print("Please upload the image you want to run the inference on")
    from google.colab import files
    src = list(files.upload().values())[0]

#### By copy from Google Drive

Be sure to customize the 2 variables for Google Colab to be able find your file in Google Drive.
Let's say you have this hierarchy in your Google Drive:
```
Root directory of Google Drive
  ├─── Directory1
  └─── Directory2
       ├─── images
       │    ├─── example1.png
       │    └─── example2.png
       └─── saved_weights
            └─── weights.h5
```
1.   ```customPathInDrive``` must represent all the directories between the root directory and your image file. In the example, it would be ```Directory2/images/```. **Do not forget the final /** if you have to use this variable;
2.   ```imageFileName``` must represent the file you want to upload. In the example, it would be ```example1.png```. It can also be empty, if you want to import all the folder's images *(and annotations files if checkbox is checked)* directly to Google Colab, so in the example ```example1.png``` and ```example2.png``` would be imported.

Use the text fields available on the right.

In [ ]:
if IN_COLAB and howToGetImage == "From Google Drive":
    pathToDrive = "'/content/drive/My Drive/"
    # Keep customPathInDrive empty if file directly in root directory of Google Drive
    customPathInDrive = "" #@param {type:"string"}
    imageFilePath = "" #@param{type:"string"}
    annotationsFile = True #@param {type:"boolean"}
    !rm -r images/ || true
    !mkdir -p images
    if imageFilePath != "":
        pathToImage = pathToDrive + customPathInDrive + imageFilePath + "'"
        tempPath = "images/" + imageFilePath
        print("Copying {} to {}".format(pathToImage, tempPath))
        !cp -u $pathToImage $tempPath
        if annotationsFile:
            annotationsFileName = imageFilePath.split('.')[0] + '.xml'
            pathToAnnotations = pathToDrive + customPathInDrive + annotationsFileName + "'"
            tempPath = "images/" + annotationsFileName
            print("Copying {} to {}".format(pathToAnnotations, tempPath))
            !cp -u $pathToAnnotations $tempPath
    else:
        pathToImageFolder = "/content/drive/My Drive/" + customPathInDrive
        fileList = os.listdir(pathToImageFolder)
        ext = ['jp2', 'png']
        if annotationsFile:
          ext.extend(['xml', 'json'])
        for dataFile in fileList:
          if dataFile.split('.')[-1] in ext:
            pathToFile = pathToDrive + customPathInDrive + dataFile + "'"
            tempPath = 'images/' + dataFile
            print("Copying {} to {}".format(pathToFile, tempPath))
            !cp -u $pathToFile $tempPath

### Retrieving Weights File

Same thing than retrieving an image file using Google Drive but it is the saved weights file (```.h5``` extension). With the past example, it would be ```Directory2/saved_weights/``` as ```customPathInDrive``` and ```weights.h5``` as ```weightFileName```.

In [ ]:
if IN_COLAB:
    pathToDrive = "'/content/drive/My Drive/"
    # Keep customPathInDrive empty if file directly in root directory of Google Drive
    customPathInDrive = "" #@param {type:"string"}
    weightFileName = "mask_rcnn_nephrologie_753_100.h5" #@param {type:"string"}
    pathToWeights = pathToDrive + customPathInDrive + weightFileName + "'"
    print("Copying {} to {}".format(pathToWeights, weightFileName))
    !cp -u $pathToWeights $weightFileName

## Initialisation

Be sure to set ```MODEL_PATH``` to the same value than ```weightFileName```. If you want to save the results in files ```saveResults``` should be checked. You will have to open the **Files tab** in the **vertical navigation bar on the left** to see the results appearing. Then you can save them by right-clicking on each file and save it. You can also customize the division size but staying at 1024 should be fine.

In [ ]:
MODEL_PATH = "mask_rcnn_nephrologie_753_100.h5" #@param {type:"string"}
RESULTS_PATH = "results/inference/" #@param {type:"string"}
DIVISION_SIZE = 1024 #@param {type:"slider", min:896, max:1024, step:1}
saveResults = True #@param {type:"boolean"}
classesInfo = [
    {"id": 0, "name": "Background", "color": "", "ignore": True},
    {"id": 1, "name": "tubule_sain", "color": "#ff007f", "ignore": False},
    {"id": 2, "name": "tubule_atrophique", "color": "#55557f", "ignore": False},
    {"id": 3, "name": "nsg_complet", "color": "#ff557f", "ignore": False},
    {"id": 4, "name": "nsg_partiel", "color": "#55aa7f", "ignore": False},
    {"id": 5, "name": "pac", "color": "#ffaa7f", "ignore": False},
    {"id": 6, "name": "vaisseau", "color": "#55ff7f", "ignore": False},
    {"id": 7, "name": "artefact", "color": "#000000", "ignore": False},
    {"id": 8, "name": "veine", "color": "#0000ff", "ignore": False},
    {"id": 9, "name": "nsg", "color": "#55007f", "ignore": True},
    {"id": 10, "name": "intima", "color": "#aa0000", "ignore": True},
    {"id": 11, "name": "media", "color": "#aa5500", "ignore": True}
]

# Following variables are used to fuse overlapping masks of the same class
# These thresholds determine the least part represented by the overlapping area
# of bounding boxes or masks to determine whether or not we should fuse two masks
FUSION_BB_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
FUSION_MASK_THRESHOLD = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

# Following variables are used to filter the masks before displaying results
# Same thing than fusion thresholds but for removing masks
FILTER_BB_THRESHOLD = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
FILTER_MASK_THRESHOLD = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
# Rows and columns correspond to classes in the same order than the classesInfo array without the first element
# An element set to true means that a mask of the column class, if contained by a mask of the
# row class, will be erased.
PRIORITY_TABLE = [[False, False, False, False, False, False, False, False],
                  [False, False, False, False, False, False, False, False],
                  [True,  True,  False, False, True,  True,  False, True],
                  [True,  True,  False, False, True,  True,  False, True],
                  [True,  True,  False, False, False, False, False, False],
                  [True,  True,  False, False, False, False, False, False],
                  [False, False, False, False, False, False, False, False],
                  [True,  True,  False, False, False, False, False, False]]

In [ ]:
import nephrology as nephro
images = nephro.listAvailableImage('images/')
print("Found {} image{}:".format(len(images), "s" if len(images) > 1 else ''))
for image in images:
    print(" -", image)

In [ ]:
model = nephro.NephrologyInferenceModel(classesInfo, MODEL_PATH, DIVISION_SIZE)
model.inference(images, RESULTS_PATH, saveResults,
                FUSION_BB_THRESHOLD, FUSION_MASK_THRESHOLD,
                FILTER_BB_THRESHOLD, FILTER_MASK_THRESHOLD, PRIORITY_TABLE,
                displayOnlyAP=False)

In [ ]:
import shutil
shutil.rmtree('data/', ignore_errors=True)